# Building Your Own Constraints and Search

In this exercice, we will show how to build additional constraints in miniCP. We will also build a custom search procedure.


## Circuit Constraint

In the previous exercice about the Dial-A-Ride Problem, we used miniCP's Circuit constraint. We will now implement our own version of this constraint, and substitute it in the CP model.

Remember that the Circuit constraint to enforce a hamiltonian circuit on a successor array. 
On the next example the successor array is `[2,4,1,5,3,0]`

![Illustration of a hamiltonian circuit](https://minicp.readthedocs.io/en/latest/_images/circuit.svg)

All the successors must be different.
But enforcing the `allDifferent` constraint is not enough.
We must also guarantee it forms a proper circuit (without sub-tours).
This can be done efficiently and incrementally by keeping track of the sub-chains
appearing during the search.
The data-structure for the sub-chains should be a reversible.
Our instance variables used to keep track of the sub-chains are:

```java
IntVar [] x;
StateInt [] dest;
StateInt [] orig;
StateInt [] lengthToDest;
```


* `dest[i]` is the furthest node we can reach from node `i` following the instantiated edges.
* `orig[i]` is the furthest node we can reach from node `i` following instantiated edges in reverse direction.
* `lengthToDest[i]` is the number of instantiated edges on the path from node `i` to `dest[i]`.

Consider the following example with instantiated edges colored in grey.

![Circuit subtour](https://minicp.readthedocs.io/en/latest/_images/circuit-subtour.svg)

Before the addition of the green link we have

```java
dest = [2,1,2,5,5,5];
orig = [0,1,0,4,4,4];
lengthToDest = [1,0,0,1,2,0];
```

After the addition of the green link we have

```java
dest = [2,1,2,2,2,2];
orig = [4,1,4,4,4,4];
lengthToDest = [1,0,0,3,4,2];
```

In your implementation you must update the reversible integers to reflect
the change after the addition of every new edge.
You can use the `CPIntVar.whenBind(...)` method for that.

The filtering in itself consists in preventing to close a
sub-tour that would have a length less than `n` (the number of nodes).
Since node 4 has a length to destination (node 2) of 4 (<6), the destination node 2 can not have 4 as successor
and the red link is deleted.
This filtering was introduced in [TSP1998] for solving the TSP with CP.


**[TSP1998]** Pesant, G., Gendreau, M., Potvin, J. Y., & Rousseau, J. M. (1998). An exact constraint logic programming algorithm for the traveling salesman problem with time windows. Transportation Science, 32(1), 12-29.

### Implementation

In miniCP, a constraint must extend the AbstractConstraint class, which has the following API:

* `void post()`: called once when the constraint is initialized. It should state the constraint and hook up the variables' callbacks (e.g. [`propagateOnDomainChainge(...)`](https://minicp.bitbucket.io/apidocs/minicp/engine/core/IntVar.html#propagateOnDomainChange-minicp.engine.core.Constraint-) or [`whenBind(...)`](https://minicp.bitbucket.io/apidocs/minicp/engine/core/IntVar.html#whenBind-minicp.util.Procedure-)),
* `void propagate()`: called by the fix-point algorithm during the search. It should remove the inconsistent values from the domain of the associated variables.


The skeleton for the implementation of a circuit constraint is given in the next code cell. Complete the missing code, then check your custom constraint in the DARP model below.

In [1]:
%jars ./

import minicp.engine.core.AbstractConstraint;
import minicp.engine.core.IntVar;
import minicp.state.StateInt;
import minicp.util.exception.NotImplementedException;

import static minicp.cp.Factory.allDifferent;

/**
 * Hamiltonian Circuit Constraint with a successor model
 */
public class MyCircuit extends AbstractConstraint {

    private final IntVar[] x;
    private final StateInt[] dest;
    private final StateInt[] orig;
    private final StateInt[] lengthToDest;

    /**
     * Creates an Hamiltonian Circuit Constraint
     * with a successor model.
     *
     * @param x the variables representing the successor array that is
     *          x[i] is the city visited after city i
     */
    public MyCircuit(IntVar[] x) {
        super(x[0].getSolver());
        assert (x.length > 0);
        this.x = x;
        dest = new StateInt[x.length];
        orig = new StateInt[x.length];
        lengthToDest = new StateInt[x.length];
        for (int i = 0; i < x.length; i++) {
            dest[i] = getSolver().getStateManager().makeStateInt(i);
            orig[i] = getSolver().getStateManager().makeStateInt(i);
            lengthToDest[i] = getSolver().getStateManager().makeStateInt(0);
        }
    }

    @Override
    public void post() {
        getSolver().post(allDifferent(x));
        // TODO
        // Hint: use x[i].whenBind(...) to call the bind(int) method
        throw new NotImplementedException("MyCircuit");
    }

    private void bind(int i) {
        // TODO
        throw new NotImplementedException("MyCircuit");
    }
}


In [2]:
%jars ./
//********************************************
//* miniPC model for the Dial-A-Ride Problem *
//* using a custom constraint for Circuit    *
//********************************************

import minicp.engine.constraints.Circuit;
import minicp.engine.constraints.Element1DVar;
import minicp.engine.core.IntVar;
import minicp.engine.core.Solver;
import minicp.search.DFSearch;
import minicp.util.Procedure;
import minicp.search.SearchStatistics;

import static minicp.cp.BranchingScheme.*;
import static minicp.cp.Factory.*;

import java.util.Arrays;
import java.util.stream.IntStream;

public static IntVar elementVar(IntVar[] array, IntVar y) {
    Solver cp = y.getSolver();
    int min = IntStream.range(0, array.length).map(i -> array[i].min()).min().getAsInt();
    int max = IntStream.range(0, array.length).map(i -> array[i].max()).max().getAsInt();
    IntVar z = makeIntVar(cp, min,max);
    cp.post(new Element1DVar(array, y, z));
    return z;
}

// DARP instance definition

// coordinates of the positions involved in this problem
int [][] coords = new int[][] {
        {10,20}, // 0
        {30,30}, // 1
        {50,40}, // 2
        {40,10}, // 3
        {50,20}, // 4
        {25,30}, // 5
        {10,5}   // 6
};

// compute the symmetrical transition times as Euclidian distances
int [][] transitions = new int[coords.length][coords.length];
for (int i = 0; i < coords.length; i++) {
    for (int j = 0; j < coords.length; j++) {
        int x1 = coords[i][0];
        int y1 = coords[i][1];
        int x2 = coords[j][0];
        int y2 = coords[j][1];
        transitions[i][j] = (int) Math.sqrt(Math.pow(x1-x2,2.0)+Math.pow(y1-y2,2.0));
    }
    System.out.println(Arrays.toString(transitions[i]));
}

// the requests defined as {fromPos,toPos,deadline}
int [][] requests = new int[][] {
        {3,5,120},
        {6,2,200},
        {5,1,100},
        {1,6,60},
        {5,2,150},
        {6,3,150},
};

// capacity of the vehicle
int vehicleCapacity = 2;

// CP model: visits, prev and succ

Solver cp = makeSolver();

int n = requests.length;
int [] v = new int[2*n+1];
v[0] = 0; // departure from the depot (position index 0)
// [1..n+1] = pickups
// [n+1..2n+1] = delivery
for (int i = 0; i < n; i++) {
    v[1+i] = requests[i][0];
    v[n+1+i] = requests[i][1];
}

IntVar [] succ = makeIntVarArray(cp, 2*n+1,2*n+1);
IntVar [] pred = makeIntVarArray(cp, 2*n+1,2*n+1);

// channeling between pred and succ vectors
for (int i = 0; i < succ.length; i++) {
    // succ[pred[i]] == i
    cp.post(equal(elementVar(succ,pred[i]),i));
}

//*******************************************
//* NOTE: here we use our custom constraint *
//*******************************************
cp.post(new MyCircuit(pred));

// CP model: time constraints

IntVar [] time = makeIntVarArray(cp, 2*n+1,500);
// departure time from the depot =  0
cp.post(equal(time[0],0));

// visit time update
for (int i = 1; i < 2*n+1; i++) {
    // time[i] = time[pred[i]] + transition[pred[i]][i]
    IntVar tPredi = elementVar(time, pred[i]);
    IntVar posPredi = element(v, pred[i]);
    IntVar tt = element(transitions[v[i]],posPredi);
    cp.post(equal(time[i],sum(tPredi,tt)));
}

// precedence between pickup and delivery + deadlines
for (int i = 0; i < n; i++) {
    // pickup before delivery)
    cp.post(lessOrEqual(time[i+1],time[n+i+1]));
    // delivery before the deadline
    cp.post(lessOrEqual(time[n+i+1],requests[i][2]));
}

// CP model: load constraints

IntVar [] load = makeIntVarArray(cp, 2*n+1,vehicleCapacity+1);
// initial load of the vehicle = 0
cp.post(equal(load[0],0));

// load update after a pickup (+1)
for (int i = 1; i <= n; i++) {
    // load[i] = load[[pred[i]] + 1
    IntVar loadPred = elementVar(load, pred[i]);
    cp.post(equal(load[i], plus(loadPred, 1)));
}

// load update after a delivery (-1)
for (int i = n+1; i <= 2*n; i++) {
    // load[i] = load[[pred[i]] - 1
    IntVar loadPred = elementVar(load, pred[i]);
    cp.post(equal(load[i], plus(loadPred, -1)));
}

[0, 22, 44, 31, 40, 18, 15]
[22, 0, 22, 22, 22, 5, 32]
[44, 22, 0, 31, 20, 26, 53]
[31, 22, 31, 0, 14, 25, 30]
[40, 22, 20, 14, 0, 26, 42]
[18, 5, 26, 25, 26, 0, 29]
[15, 32, 53, 30, 42, 29, 0]


In [3]:
DFSearch dfs = makeDfs(cp, firstFail(pred));

dfs.onSolution(() -> {
    System.out.println("solution");
    System.out.println("succ:"+Arrays.toString(succ));
    System.out.println("pred:"+Arrays.toString(pred));
    System.out.println("time:"+Arrays.toString(time));
    System.out.println("load:"+Arrays.toString(load));

    int curr = 0;
    for (int i = 0; i < succ.length; i++) {
        System.out.println("visiting position:"+v[curr]+" at time:"+time[curr]+" load:"+load[curr]);
        curr = succ[curr].min();
    }

});

// search for the first feasible solution
SearchStatistics stats = dfs.solve(statistics -> {
    return statistics.numberOfSolutions() > 0;
});

System.out.println(stats);

solution
succ:[{3}, {7}, {10}, {9}, {2}, {8}, {12}, {5}, {11}, {4}, {6}, {0}, {1}]
pred:[{11}, {12}, {4}, {0}, {9}, {7}, {10}, {1}, {5}, {3}, {2}, {8}, {6}]
time:[{0}, {85}, {55}, {18}, {23}, {110}, {55}, {110}, {136}, {23}, {55}, {136}, {85}]
load:[{0}, {2}, {2}, {1}, {1}, {2}, {2}, {1}, {1}, {0}, {1}, {0}, {1}]
visiting position:0 at time:{0} load:{0}
visiting position:5 at time:{18} load:{1}
visiting position:1 at time:{23} load:{0}
visiting position:1 at time:{23} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:6 at time:{55} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:3 at time:{85} load:{1}
visiting position:3 at time:{85} load:{2}
visiting position:5 at time:{110} load:{1}
visiting position:5 at time:{110} load:{2}
visiting position:2 at time:{136} load:{1}
visiting position:2 at time:{136} load:{0}

	#choice: 31854
	#fail: 15923
	#sols : 1
	completed : false



## Custom Search for DARP

To write a custom search in miniCP, you can make use of the method [`Factory.makeDfs`](https://minicp.bitbucket.io/apidocs/minicp/cp/Factory.html#makeDfs-minicp.engine.core.Solver-java.util.function.Supplier-). It takes two parameters:
* `Solver cp` the solver that will be used for the search
* `Supplier<Procedure[]> branching` a generator that is called at each node of the depth first search tree to generate an array of [`Procedure`](https://minicp.bitbucket.io/apidocs/minicp/util/Procedure.html) objects that will be used to commit to child nodes. It should return `BranchingScheme#EMPTY` whenever the current state is a solution.

Example of binary search: At each node it selects the first free variable `predi` from the array `pred`, and creates two branches `predi=v`, `predi!=v` where `v` is the min value domain

```java
DFSearch myDfs = makeDfs(cp, () -> {
    Optional<IntVar> predi = Arrays.stream(pred)
        .filter(b -> b.size() > 1) // only select free variables
        .findFirst(); // select the first one
    if (predi.isEmpty()) {
        return EMPTY;
    } else {
        int v = predi.get().min();
        Procedure left = () -> cp.post(equal(predi.get(), v)); // left branch
        Procedure right = () -> cp.post(notEqual(predi.get(), v)); // right branch
        return branch(left, right);
    }
});
```

The branching scheme used in our DARP model is called [`firstFail`](https://minicp.bitbucket.io/apidocs/minicp/cp/BranchingScheme.html#firstFail-minicp.engine.core.IntVar...-), and does this with the addition that the variable with the smallest domain is selected first for branching. The value selection strategy (always taking the min value from the domain) is not well suited for the DARP. The one you design should be more similar to the decision you would make manually in a greedy fashion. For instance you can explore first the branch `pred[i] = x` where `x` is the closest location to `i` in the domain.

You can also implement a min-regret variable selection strategy. It selects the variable with the largest different between the closest predecessor city and the second closest one. The idea is that it is critical to decide the predecessor for this city first because otherwise you will regret it the most.

Hint: Since there is no iterator on the domain of a variable, you can iterate from the minimum value to the maximum one using a for loop and check if it is in the domain with the contains method.

Implement a custom DFS search in the code block below. Observe the first solution obtained and its objective value ? Is it better than the naive first fail ? Also observe the time and number of backtracks necessary for proving optimality. By how much did you reduce the computation time ?

In [8]:
DFSearch myDfs = makeDfs(cp, () -> {
    Optional<IntVar> predi = Arrays.stream(pred)
        .filter(b -> b.size() > 1) // only select free variables
        .min((a,b) -> a.size()-b.size()); // select the one with smallest domain
    if (predi.isEmpty()) {
        return EMPTY;
    } else {
        int v = predi.get().min(); // CHANGE THIS
        Procedure left = () -> cp.post(equal(predi.get(), v)); // left branch
        Procedure right = () -> cp.post(notEqual(predi.get(), v)); // right branch
        return branch(left, right);
    }
});

myDfs.onSolution(() -> {
    System.out.println("solution");
    System.out.println("succ:"+Arrays.toString(succ));
    System.out.println("pred:"+Arrays.toString(pred));
    System.out.println("time:"+Arrays.toString(time));
    System.out.println("load:"+Arrays.toString(load));

    int curr = 0;
    for (int i = 0; i < succ.length; i++) {
        System.out.println("visiting position:"+v[curr]+" at time:"+time[curr]+" load:"+load[curr]);
        curr = succ[curr].min();
    }

});

// search for the first feasible solution
SearchStatistics stats = myDfs.solve(statistics -> {
    return statistics.numberOfSolutions() > 0;
});

System.out.println(stats);

solution
succ:[{3}, {7}, {10}, {9}, {2}, {8}, {12}, {5}, {11}, {4}, {6}, {0}, {1}]
pred:[{11}, {12}, {4}, {0}, {9}, {7}, {10}, {1}, {5}, {3}, {2}, {8}, {6}]
time:[{0}, {85}, {55}, {18}, {23}, {110}, {55}, {110}, {136}, {23}, {55}, {136}, {85}]
load:[{0}, {2}, {2}, {1}, {1}, {2}, {2}, {1}, {1}, {0}, {1}, {0}, {1}]
visiting position:0 at time:{0} load:{0}
visiting position:5 at time:{18} load:{1}
visiting position:1 at time:{23} load:{0}
visiting position:1 at time:{23} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:6 at time:{55} load:{1}
visiting position:6 at time:{55} load:{2}
visiting position:3 at time:{85} load:{1}
visiting position:3 at time:{85} load:{2}
visiting position:5 at time:{110} load:{1}
visiting position:5 at time:{110} load:{2}
visiting position:2 at time:{136} load:{1}
visiting position:2 at time:{136} load:{0}

	#choice: 31854
	#fail: 15923
	#sols : 1
	completed : false

